Even after subtracting the combinatorial background (signal =unmixed events - mixed events), we deal with a large number of backgrounds. By finding the optimum event selections, we can remove the unphysical dimuons from the signal+other Background events. To take care of that, we can compare Monte Carlo distributions and find optimum selections. This is called signal to background significance $\frac{S}{\sqrt{S+B}}$. The details study can be found in the [docdb:10221](https://seaquest-docdb.fnal.gov/cgi-bin/sso/ShowDocument?docid=10221). From the study, the track and dimuon event selections have been added to the "cutDUMPDY.h". The reconstructed events from the SeaQuest roadset-67 dataset and the reconstructed data from the Monte Carlo will get affected by the same even selections. All the seelctions will pass to the [RDataFrame Filter](https://root.cern/doc/master/classROOT_1_1RDataFrame.html). In the Filter, we have a lot selections that are added as && condition, and they are executed in order and the first one returning false causes the event to be discarded.

In [1]:
#include "TLegend.h"
#include "TLine.h"
#include "TLatex.h"
#include "TMarker.h"
#include "ana_const.h"
#include "cutDUMPDY.h"

The static function SetDefaultSumw2() is called in the beginning since we are going to deal with the weighted Histograms. Therefore the Sumw2() function is automatically called wheever any histograms is created.

In [2]:
%jsroot on
ROOT::EnableImplicitMT();
TH1::SetDefaultSumw2();
TH2::SetDefaultSumw2();
gStyle->SetPalette(1);
gStyle->SetPaintTextFormat("5.4f");
ROOT::RDataFrame dmyDf_mc("result_mc", "combined_LD2_DUMP_67_GMC.root");  //LD2 random hit-embedded MC
ROOT::RDataFrame dmyDf_data("result", "merged_RS67_3087all_target.root");  //All Target combined unmixed reconstructed toadset-67 events 
ROOT::RDataFrame dmyDf_mix("result_mix", "merged_RS67_3087all_target.root");//All Target combined mixed reconstructed toadset-67 events 
ROOT::RDataFrame dmyDf_data_ld2("result", "merged_RS67_3089LD2.root");  //All Target combined LD2 unmixed reconstructed toadset-67 events 
ROOT::RDataFrame dmyDf_mix_ld2("result_mix", "merged_RS67_3089LD2.root");//All Target combined LD2 mixed reconstructed toadset-67 events 

TH1D* h_pt = new TH1D("h_pt","h_pt", nBinpt, ptMin, ptMax);
TH1D* h_x2 = new TH1D("h_x2","h_x2", nBinx2, x2Min, x2Max);
gROOT->ForceStyle();
//gStyle->SetOptStat("0");
gStyle->SetOptFit(1111);
//TCanvas* c1 = new TCanvas("c1","c1",600,600);
TCanvas *c1[4];
TFile *myfile = new TFile("phi_costh.root","recreate");
TF2* fit2D = new TF2("fit2D", "[0] * ( 1 + [1]*y*y + 2*[2]*sqrt(1-y*y)*y*cos(x) + [3]*(1-y*y)*cos(2*x)/2.) ", -M_PI, M_PI, -0.5, 0.5);
fit2D->SetParameters(1,1,0,0);
fit2D->SetParNames("A", "#lambda","#mu","#nu");

Angular distribution coefficients in the following conditions:

        1. Data: All Mixed target data.
        2. GMC : LD2 random hit embedded Drell--Yan 
        3. D1 < 100
        4. 4 Dimuon pT bins in the range (0 to 2.0 GeV)
       

In [3]:
for (int i_xT = 1; i_xT <=nBinx2 ; i_xT++) {
    for (int i_pt = 1; i_pt <=nBinpt ; i_pt++) {
        Double_t xBinMinPt = h_pt->GetXaxis()->GetBinLowEdge(i_pt);
        Double_t xBinMaxPt = h_pt->GetXaxis()->GetBinUpEdge(i_pt);
        Double_t xBinMinx2 = h_x2->GetXaxis()->GetBinLowEdge(i_xT);
        Double_t xBinMax2  = h_x2->GetXaxis()->GetBinUpEdge(i_xT);
        //RDATAFRAME
        string  ptCutTrue = Form("true_pT>%4.10f&&true_pT<%4.10f&&",xBinMinPt,xBinMaxPt);
        string trigCut ="fpga1==1&&";
        string massCut = "true_mass>4.5 && true_mass<8.0";
        string occu2 ="D1<100";
        string dumpCutRD= pos+neg+dimu+occu2;
        string filter_mc_4pi=ptCutTrue+massCut; // 4 pi mc filter 
        string filter_mc_reco=ptCutTrue+trigCut+massCut+"&&"+dumpCutRD; // reco mc filter
        //Data
        string pTBins= Form("pT>%4.10f&&pT<%4.10f&&",xBinMinPt,xBinMaxPt);
        string xTBins=  Form("xT>%4.10f&&xT<%4.10f&&",xBinMinx2,xBinMax2);
        string massRange=" mass>4.5 && mass<8.0&&";
        string final_cut_filter=pTBins+xTBins+massRange+dumpCutRD; // data filter 

        auto hist2d_unmix = dmyDf_data.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_unmix", "phi_theta_unmix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax}, "phi_", "costh_");
        auto hist2d_mix = dmyDf_mix.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_mix", "phi_theta_mix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_", "costh_");

        auto hist2d_mc_4pi = dmyDf_mc.Filter(filter_mc_4pi).Define("phi_mc_4pi","true_phi").Define("costh_mc_4pi","true_costh").Histo2D({"phi_theta_mc_4pi", "phi_theta_mc_4pi", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_mc_4pi", "costh_mc_4pi","weight");
        auto hist2d_mc_reco = dmyDf_mc.Filter(filter_mc_reco).Define("phi_mc_reco","phi").Define("costh_mc_reco","costh").Histo2D({"phi_theta_mc_reco", "phi_theta_mc_reco", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_mc_reco", "costh_mc_reco","weight");
        c1[i_pt] = new TCanvas(Form("c%i",i_pt));
        c1[i_pt]->Divide(2,2);
        hist2d_unmix->Add(hist2d_mix.GetPtr(),-1);
        cout <<"Integrated signal count: "<< hist2d_unmix->Integral()<<endl;
        hist2d_unmix->SetName(Form("phi_theta_ipT%i",i_pt));
        TH2*h2_phi_costh = (TH2D*)hist2d_unmix->Clone();
        c1[i_pt]->cd(1);
        h2_phi_costh->Draw("colz text");
        h2_phi_costh->SetTitle(Form("(Unmix -Mix) Events in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));
        //h2_phi_costh->SetStats(0);
        c1[i_pt]->cd(2);
        //gStyle->SetPaintTextFormat("5.4f");
        gStyle->SetPaintTextFormat("4.1f m");
        TH2*acc_factor = (TH2D*)hist2d_mc_reco->Clone();
        acc_factor->SetName(Form("acc_factor_ipT%i",i_pt));
        acc_factor->Divide(hist2d_mc_4pi.GetPtr());
        //acc_factor->SetStats(0);
        acc_factor->SetTitle(Form("Acceptance factor in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));

        acc_factor->Draw("colz text");
        //gStyle->SetOptStat(0);
        c1[i_pt]->cd(3);
        TH2*acc_corrected_h2d = (TH2D*)h2_phi_costh->Clone();
        acc_corrected_h2d->Divide(acc_factor);
        acc_corrected_h2d->Fit("fit2D");
        acc_corrected_h2d->SetTitle(Form("Acc. corrected in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));
        acc_corrected_h2d->Draw("colz");
        c1[i_pt]->SaveAs(Form("phi_theta_ipT_%i.png",i_pt));
        c1[i_pt]->Draw();
        //c1[i_pt]->Updated();
    }
}

Integrated signal count: 179930
 FCN=157.669 FROM MIGRAD    STATUS=CONVERGED     852 CALLS         853 TOTAL
                     EDM=9.70472e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   3.9 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            1.60633e+05   4.39382e+03   6.42075e+01   2.52334e-09
   2  #lambda     -5.13422e-02   2.97337e-01  -4.85761e-03   5.09111e-03
   3  #mu          1.60272e-01   6.43871e-02  -9.77664e-04  -1.15989e-02
   4  #nu          7.53293e-02   5.89911e-02  -7.40490e-04  -1.80512e-02
Integrated signal count: 269305
 FCN=154.19 FROM MIGRAD    STATUS=CONVERGED     117 CALLS         118 TOTAL
                     EDM=1.12001e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            2.89157e+05

Info in <TCanvas::Print>: png file phi_theta_ipT_1.png has been created
Info in <TCanvas::Print>: png file phi_theta_ipT_2.png has been created
Info in <TCanvas::Print>: png file phi_theta_ipT_3.png has been created
Info in <TCanvas::Print>: png file phi_theta_ipT_4.png has been created


Angular distribution coefficients in the following conditions:

        1. Data: All Mixed target data.
        2. GMC : LD2 random hit embedded Drell--Yan.
        3. 100< D1 < 200.
        4. 4 Dimuon pT bins in the range (0 to 2.0 GeV)
       

In [4]:
for (int i_xT = 1; i_xT <=nBinx2 ; i_xT++) {
    for (int i_pt = 1; i_pt <=nBinpt ; i_pt++) {
        Double_t xBinMinPt = h_pt->GetXaxis()->GetBinLowEdge(i_pt);
        Double_t xBinMaxPt = h_pt->GetXaxis()->GetBinUpEdge(i_pt);
        Double_t xBinMinx2 = h_x2->GetXaxis()->GetBinLowEdge(i_xT);
        Double_t xBinMax2 = h_x2->GetXaxis()->GetBinUpEdge(i_xT);
        //RDATAFRAME
        string  ptCutTrue = Form("true_pT>%4.10f&&true_pT<%4.10f&&",xBinMinPt,xBinMaxPt);
        string trigCut ="fpga1==1&&";
        string massCut = "true_mass>4.5 && true_mass<8.0";
        string occu2 ="100<D1 and D1<200";
        string dumpCutRD= pos+neg+dimu+occu2;
        string filter_mc_4pi=ptCutTrue+massCut; // 4 pi mc filter 
        string filter_mc_reco=ptCutTrue+trigCut+massCut+"&&"+dumpCutRD; // reco mc filter
        //Data
        string pTBins= Form("pT>%4.10f&&pT<%4.10f&&",xBinMinPt,xBinMaxPt);
        string xTBins=  Form("xT>%4.10f&&xT<%4.10f&&",xBinMinx2,xBinMax2);
        string massRange=" mass>4.5 && mass<8.0&&";
        string final_cut_filter=pTBins+xTBins+massRange+dumpCutRD; // data filter 

        auto hist2d_unmix = dmyDf_data.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_unmix", "phi_theta_unmix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax}, "phi_", "costh_");
        auto hist2d_mix = dmyDf_mix.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_mix", "phi_theta_mix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_", "costh_");

        auto hist2d_mc_4pi = dmyDf_mc.Filter(filter_mc_4pi).Define("phi_mc_4pi","true_phi").Define("costh_mc_4pi","true_costh").Histo2D({"phi_theta_mc_4pi", "phi_theta_mc_4pi", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_mc_4pi", "costh_mc_4pi","weight");
        auto hist2d_mc_reco = dmyDf_mc.Filter(filter_mc_reco).Define("phi_mc_reco","phi").Define("costh_mc_reco","costh").Histo2D({"phi_theta_mc_reco", "phi_theta_mc_reco", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_mc_reco", "costh_mc_reco","weight");
        c1[i_pt] = new TCanvas(Form("c%i",i_pt));
        c1[i_pt]->Divide(2,2);
        hist2d_unmix->Add(hist2d_mix.GetPtr(),-1);
        cout <<"Integrated signal count: "<< hist2d_unmix->Integral()<<endl;
        hist2d_unmix->SetName(Form("phi_theta_ipT%i",i_pt));
        TH2*h2_phi_costh = (TH2D*)hist2d_unmix->Clone();
        c1[i_pt]->cd(1);
        h2_phi_costh->Draw("colz text");
        h2_phi_costh->SetTitle(Form("(Unmix -Mix) Events in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));
        //h2_phi_costh->SetStats(0);
        c1[i_pt]->cd(2);
        //gStyle->SetPaintTextFormat("5.4f");
        gStyle->SetPaintTextFormat("4.1f m");
        TH2*acc_factor = (TH2D*)hist2d_mc_reco->Clone();
        acc_factor->SetName(Form("acc_factor_ipT%i",i_pt));
        acc_factor->Divide(hist2d_mc_4pi.GetPtr());
        //acc_factor->SetStats(0);
        acc_factor->SetTitle(Form("Acceptance factor in pT_{low}=%4.2f and pT_{high}=%4.2f;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));

        acc_factor->Draw("colz text");
        //gStyle->SetOptStat(0);
        c1[i_pt]->cd(3);
        TH2*acc_corrected_h2d = (TH2D*)h2_phi_costh->Clone();
        acc_corrected_h2d->Divide(acc_factor);
        acc_corrected_h2d->Fit("fit2D");
        acc_corrected_h2d->SetTitle(Form("Acc. corrected in pT_{low}=%4.2f and pT_{high}=%4.2f;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));
        acc_corrected_h2d->Draw("colz");
        c1[i_pt]->SaveAs(Form("phi_theta_ipT_%i.png",i_pt));
        c1[i_pt]->Draw();
        //c1[i_pt]->Updated();
    }
}

Integrated signal count: 241250
 FCN=153.064 FROM MIGRAD    STATUS=CONVERGED     125 CALLS         126 TOTAL
                     EDM=1.87921e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.9 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            1.45789e+05   3.25291e+03  -1.02081e+01   5.47147e-09
   2  #lambda     -6.17045e-02   2.42657e-01   3.95282e-04  -6.09818e-04
   3  #mu          9.30018e-02   5.34335e-02   9.64000e-05   1.47194e-03
   4  #nu          2.42724e-01   4.81396e-02   3.51069e-05   4.41175e-03
Integrated signal count: 374882
 FCN=262.006 FROM MIGRAD    STATUS=CONVERGED      92 CALLS          93 TOTAL
                     EDM=8.28149e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.3 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A         

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file phi_theta_ipT_1.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Info in <TCanvas::Print>: png file phi_theta_ipT_2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Info in <TCanvas::Print>: png file phi_theta_ipT_3.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Info in <TCanvas::Print>: png file phi_theta_ipT_4.png has been created


Angular distribution coefficients in the following conditions:

        1. Data: LD2 target data.
        2. GMC : LD2 random hit embedded Drell--Yan.
        3. D1 < 200.
        4. 4 Dimuon pT bins in the range (0 to 2.0 GeV)
       

In [6]:
for (int i_xT = 1; i_xT <=nBinx2 ; i_xT++) {
    for (int i_pt = 1; i_pt <=nBinpt ; i_pt++) {
        Double_t xBinMinPt = h_pt->GetXaxis()->GetBinLowEdge(i_pt);
        Double_t xBinMaxPt = h_pt->GetXaxis()->GetBinUpEdge(i_pt);
        Double_t xBinMinx2 = h_x2->GetXaxis()->GetBinLowEdge(i_xT);
        Double_t xBinMax2  = h_x2->GetXaxis()->GetBinUpEdge(i_xT);
        //RDATAFRAME
        string  ptCutTrue = Form("true_pT>%4.10f&&true_pT<%4.10f&&",xBinMinPt,xBinMaxPt);
        string trigCut ="fpga1==1&&";
        string massCut = "true_mass>4.5 && true_mass<8.0";
        string occu2 ="D1<200";
        string dumpCutRD= pos+neg+dimu+occu2;
        string filter_mc_4pi=ptCutTrue+massCut; // 4 pi mc filter 
        string filter_mc_reco=ptCutTrue+trigCut+massCut+"&&"+dumpCutRD; // reco mc filter
        //Data
        string pTBins= Form("pT>%4.10f&&pT<%4.10f&&",xBinMinPt,xBinMaxPt);
        string xTBins=  Form("xT>%4.10f&&xT<%4.10f&&",xBinMinx2,xBinMax2);
        string massRange=" mass>4.5 && mass<8.0&&";
        string final_cut_filter=pTBins+xTBins+massRange+dumpCutRD; // data filter 

        auto hist2d_unmix = dmyDf_data_ld2.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_unmix", "phi_theta_unmix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax}, "phi_", "costh_");
        auto hist2d_mix = dmyDf_mix_ld2.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_mix", "phi_theta_mix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_", "costh_");

        auto hist2d_mc_4pi = dmyDf_mc.Filter(filter_mc_4pi).Define("phi_mc_4pi","true_phi").Define("costh_mc_4pi","true_costh").Histo2D({"phi_theta_mc_4pi", "phi_theta_mc_4pi", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_mc_4pi", "costh_mc_4pi","weight");
        auto hist2d_mc_reco = dmyDf_mc.Filter(filter_mc_reco).Define("phi_mc_reco","phi").Define("costh_mc_reco","costh").Histo2D({"phi_theta_mc_reco", "phi_theta_mc_reco", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_mc_reco", "costh_mc_reco","weight");
        c1[i_pt] = new TCanvas(Form("c%i",i_pt));
        c1[i_pt]->Divide(2,2);
        hist2d_unmix->Add(hist2d_mix.GetPtr(),-1);
        cout <<"Integrated signal count: "<< hist2d_unmix->Integral()<<endl;
        hist2d_unmix->SetName(Form("phi_theta_ipT%i",i_pt));
        TH2*h2_phi_costh = (TH2D*)hist2d_unmix->Clone();
        c1[i_pt]->cd(1);
        h2_phi_costh->Draw("colz text");
        h2_phi_costh->SetTitle(Form("(Unmix -Mix) Events in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));
        //h2_phi_costh->SetStats(0);
        c1[i_pt]->cd(2);
        //gStyle->SetPaintTextFormat("5.4f");
        gStyle->SetPaintTextFormat("4.1f m");
        TH2*acc_factor = (TH2D*)hist2d_mc_reco->Clone();
        acc_factor->SetName(Form("acc_factor_ipT%i",i_pt));
        acc_factor->Divide(hist2d_mc_4pi.GetPtr());
        //acc_factor->SetStats(0);
        acc_factor->SetTitle(Form("Acceptance factor in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));

        acc_factor->Draw("colz text");
        //gStyle->SetOptStat(0);
        c1[i_pt]->cd(3);
        TH2*acc_corrected_h2d = (TH2D*)h2_phi_costh->Clone();
        acc_corrected_h2d->Divide(acc_factor);
        acc_corrected_h2d->Fit("fit2D");
        acc_corrected_h2d->SetTitle(Form("Acc. corrected in pT_{low}=%4.2f and pT_{high}=%4.2f ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt));
        acc_corrected_h2d->Draw("colz");
        c1[i_pt]->SaveAs(Form("phi_theta_ipT_%i.png",i_pt));
        c1[i_pt]->Draw();
        //c1[i_pt]->Updated();
    }
}

Integrated signal count: 75370
 FCN=186.845 FROM MIGRAD    STATUS=CONVERGED     115 CALLS         116 TOTAL
                     EDM=5.56895e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.6 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            2.76750e+04   4.94038e+02   4.83781e-01   1.32128e-07
   2  #lambda     -1.90054e-01   1.98928e-01  -1.63093e-04   1.74623e-04
   3  #mu          1.02833e-01   4.40360e-02   1.36001e-04   2.76374e-03
   4  #nu          1.81900e-01   3.84126e-02  -5.71370e-05  -1.77009e-03
Integrated signal count: 115086
 FCN=246.537 FROM MIGRAD    STATUS=CONVERGED     117 CALLS         118 TOTAL
                     EDM=6.52612e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            5.03781e+04

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file phi_theta_ipT_1.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Info in <TCanvas::Print>: png file phi_theta_ipT_2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Info in <TCanvas::Print>: png file phi_theta_ipT_3.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Info in <TCanvas::Print>: png file phi_theta_ipT_4.png has been created
